In [1]:
import numpy as np
import tensorflow as tf
from src.data_utils import Dataset
from src.network import unet_3d


BATCH_SIZE = 2
DEPTH = 3
N_BASE_FILTERS = 4
NUM_CLASSES = 3

WH = 128
MAX_DEPTH = 32

TRAIN_N = 70
TEST_N = 10
EPOCHS = 1
CLASS_WEIGHTS = np.array([0.00444413, 0.69519929, 0.30035659])

test_dataset = Dataset.load_dataset(
    '../data/processed/test_dataset.pckl'
).create_tf_dataset().padded_batch(
    batch_size=1,
    padded_shapes=([MAX_DEPTH, WH, WH, 1], [MAX_DEPTH, WH, WH, NUM_CLASSES]))

# setup train and test dataset iterator objects
iterator = tf.data.Iterator.from_structure(
    test_dataset.output_types,
    test_dataset.output_shapes
)
test_init_op = iterator.make_initializer(test_dataset)

# first element is scans, second is target, i.e. segmentation
next_element = iterator.get_next()

# batch normalisation uses population mean/var at testing
train_phase = tf.placeholder(tf.bool, name="is_training")

# create the neural network model
logits = unet_3d(
    next_element[0],
    training=train_phase,
    depth=DEPTH,
    n_base_filters=N_BASE_FILTERS,
    num_classes=NUM_CLASSES
)

# weighted softmax, see https://stackoverflow.com/a/44563055
class_weights = tf.cast(tf.constant(CLASS_WEIGHTS), tf.float32)
class_weights = tf.reduce_sum(
    tf.cast(next_element[0], tf.float32) * class_weights, axis=-1
)
loss = tf.losses.softmax_cross_entropy(
    logits=logits,
    onehot_labels=next_element[1],
    weights=class_weights
)
# as per TF batch_norm docs and also following https://goo.gl/1UVeYK
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = tf.train.AdamOptimizer().minimize(loss)

# get mean IOU (channel last tensors)
prediction = tf.argmax(logits, axis=-1)
labels = tf.argmax(next_element[1], -1)

sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph('/tmp/model.ckpt.meta')
saver.restore(sess, tf.train.latest_checkpoint('/tmp/'))
graph = tf.get_default_graph()
sess.run(tf.global_variables_initializer())
sess.run(test_init_op)
y_pred, y_true = sess.run([prediction, labels], feed_dict={train_phase: False})    

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt


In [2]:
import numpy as np
import tensorflow as tf
from src.data_utils import Dataset
from src.network import unet_3d


BATCH_SIZE = 2
DEPTH = 3
N_BASE_FILTERS = 4
NUM_CLASSES = 3

WH = 128
MAX_DEPTH = 32

TRAIN_N = 70
TEST_N = 10
EPOCHS = 1
CLASS_WEIGHTS = np.array([0.00444413, 0.69519929, 0.30035659])


test_dataset = Dataset.load_dataset(
    '../data/processed/test_dataset.pckl'
).create_tf_dataset().padded_batch(
    batch_size=1,
    padded_shapes=([MAX_DEPTH, WH, WH, 1], [MAX_DEPTH, WH, WH, NUM_CLASSES])
).make_one_shot_iterator()

sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph('/tmp/model.ckpt.meta')
saver.restore(sess, tf.train.latest_checkpoint('/tmp/'))
graph = tf.get_default_graph()

prediction = graph.get_tensor_by_name("prediction:0")
iou = graph.get_tensor_by_name("iou:0")
train_phase = graph.get_tensor_by_name("is_training:0")
top = [op for op in sess.graph.get_operations() if op.name=='test_init_op']
sess.run(top[0], feed_dict={test_dataset})
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
pred, miou = sess.run([prediction, iou], feed_dict={train_phase: False})    

/home/daniel/.virtualenvs/tf/lib/python3.5/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:Restoring parameters from /tmp/model.ckpt


TypeError: input must be a dictionary

In [12]:
plt.imshow(pred[0, 15, :, :])

NameError: name 'plt' is not defined

In [ ]:
graph.get_tensor_by_name

In [ ]:
test_init_op = graph.get_operation_by_name('IteratorV2').make_initializer(test_dataset)

In [25]:
top = [op for op in sess.graph.get_operations() if op.name=='test_init_op']
sess.run(top[0])

In [16]:
ops[0].type = 'IteratorGetNext'

'Const'